In [196]:
#import libraries
import numpy as np
import pandas
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report
from time import time
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [199]:
#import data
data_small = pandas.read_csv('data28.csv')
y = data_small['label'] #original
X = data_small[data_small.columns[1:]] #original


#newly added
#reshape into 64*264 array
#data_small.iloc[3,1:].values.reshape(28, 28).astype('uint8')
#X=data_small.iloc[:,1:].values.reshape(len(data_small),28,28,1)
#y=data_small.iloc[:,0].values

# 80-20 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# 80-20 train-validation split
X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size = 0.2, random_state = 1)

In [200]:
#preprocess

# indices_to_keep=~X_train.isin([np.nan,np.inf,-np.inf]).any(1)

# X_train=X_train[indices_to_keep]
# y_train=y_train[indices_to_keep]

indices_to_keep=~X_training.isin([np.nan,np.inf,-np.inf]).any(1)
X_training = X_training[indices_to_keep]
y_training = y_training[indices_to_keep]

indices_to_keep=~X_validation.isin([np.nan,np.inf,-np.inf]).any(1)
X_validation = X_validation[indices_to_keep]
y_validation = y_validation[indices_to_keep]

indices_to_keep_test=~X_test.isin([np.nan,np.inf,-np.inf]).any(1)

X_test=X_test[indices_to_keep_test]
y_test=y_test[indices_to_keep_test]

#new: 
# indices_to_keep=~X.isin([np.nan,np.inf,-np.inf]).any(1)
# X = X[indices_to_keep]
# y = y[indices_to_keep]

In [201]:
#newly added
# length = len(X_train)
# X_train=X_train.iloc[:,:].values.reshape(length,64,64,1)
#y_train = y_train.values

length_train = len(X_training)
X_training=X_training.iloc[:,:].values.reshape(length_train,28,28,1)

length_validation = len(X_validation)
X_validation=X_validation.iloc[:,:].values.reshape(length_validation,28,28,1)

length_t = len(X_test)
X_test = X_test.iloc[:,:].values.reshape(length_t,28,28,1)

# length = len(X)
# X = X.iloc[:,:].values.reshape(length, 28, 28, 1)

In [202]:
X.shape

(5000, 784)

In [203]:
X_training.shape

(3200, 28, 28, 1)

In [204]:
X_validation.shape

(800, 28, 28, 1)

In [205]:
X_test.shape

(1000, 28, 28, 1)

In [207]:
#newly added
#encoder = sklearn.preprocessing.LabelEncoder
#y_train = encoder.fit_transform(["Whole", "Half", "Quaters", "Eight", "Sixteenth"],y_train)
#print(y_train)
#y_test = keras.utils.to_categorical(y_test, num_classes=5)
y_training_n = []
for item in y_training:
    if item == "Whole":
        y_training_n.append(0)
    elif item == "Half":
        y_training_n.append(1)
    elif item == "Quarter":
        y_training_n.append(2)
    elif item == "Eight":
        y_training_n.append(3)
    elif item == "Sixteenth":
        y_training_n.append(4)
#print(y_train_n)
#y_train_n = np.array(y_train_n)
#print(y_train_n)
y_training_n = keras.utils.to_categorical(y_training_n, num_classes=5)
#print(y_train_n)

In [208]:
y_validation_n = []
for item in y_validation:
    if item == "Whole":
        y_validation_n.append(0)
    elif item == "Half":
        y_validation_n.append(1)
    elif item == "Quarter":
        y_validation_n.append(2)
    elif item == "Eight":
        y_validation_n.append(3)
    elif item == "Sixteenth":
        y_validation_n.append(4)
#print(y_train_n)
# y_train_n = np.array(y_train_n)
#print(y_train_n)
y_validation_n = keras.utils.to_categorical(y_validation_n, num_classes=5)

In [209]:
#newly added
y_test_n = []
for ele in y_test:
    if ele == "Whole":
        y_test_n.append(0)
    elif ele == "Half":
        y_test_n.append(1)
    elif ele == "Quarter":
        y_test_n.append(2)
    elif ele == "Eight":
        y_test_n.append(3)
    elif ele == "Sixteenth":
        y_test_n.append(4)
y_test_n = keras.utils.to_categorical(y_test_n, num_classes=5)
#print(y_test_n)

In [210]:
print(y_training_n.shape)
print(y_training.shape)
print(y_validation_n.shape)
print(y_validation.shape)
print(y_test_n.shape)
print(y_test.shape)
print(X_training.shape)
print(X_validation.shape)
print(X_test.shape)

(3200, 5)
(3200,)
(800, 5)
(800,)
(1000, 5)
(1000,)
(3200, 28, 28, 1)
(800, 28, 28, 1)
(1000, 28, 28, 1)


In [214]:
#CNN
model = Sequential()
model.add(Convolution2D(32,3,data_format="channels_last",activation="tanh",input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(40,1,data_format="channels_last",activation="tanh",input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(50,11,activation="tanh"))
model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Convolution2D(40,2,activation='relu'))
# model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Flatten())
#model.add(Dense(100))
#model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adadelta', metrics=['accuracy'])

In [215]:
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 3, 3, 40)          154920    
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 1, 1, 40)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 40)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 205       
_________________________________________________________________
activation_13 (Activation)   (None, 5)               

In [216]:
t0 = time()
model.fit(X_training, y_training_n, validation_data = (X_validation, y_validation_n), epochs=450)
#t1 = time.time()
print(time()-t0)

Epoch 1/450
100/100 [==============================] - 4s 39ms/step - loss: 1.7707 - accuracy: 0.1932 - val_loss: 1.6791 - val_accuracy: 0.1875
Epoch 2/450
100/100 [==============================] - 4s 38ms/step - loss: 1.6555 - accuracy: 0.2121 - val_loss: 1.6250 - val_accuracy: 0.1775
Epoch 3/450
100/100 [==============================] - 4s 38ms/step - loss: 1.6001 - accuracy: 0.2225 - val_loss: 1.5901 - val_accuracy: 0.2087
Epoch 4/450
100/100 [==============================] - 4s 37ms/step - loss: 1.5693 - accuracy: 0.2553 - val_loss: 1.5602 - val_accuracy: 0.2862
Epoch 5/450
100/100 [==============================] - 4s 38ms/step - loss: 1.5430 - accuracy: 0.3326 - val_loss: 1.5338 - val_accuracy: 0.3313
Epoch 6/450
100/100 [==============================] - 4s 37ms/step - loss: 1.5226 - accuracy: 0.3527 - val_loss: 1.5075 - val_accuracy: 0.3862
Epoch 7/450
100/100 [==============================] - 4s 37ms/step - loss: 1.4946 - accuracy: 0.4180 - val_loss: 1.4836 - val_accuracy:

In [219]:
model.predict(X_test)

array([[0.00554522, 0.14940096, 0.7473883 , 0.08830032, 0.00936509],
       [0.00587974, 0.01001229, 0.07384551, 0.66180265, 0.24845985],
       [0.01248588, 0.04381266, 0.11141603, 0.6098515 , 0.2224339 ],
       ...,
       [0.00317946, 0.03565574, 0.00659291, 0.20390679, 0.7506651 ],
       [0.00977425, 0.8537417 , 0.07179154, 0.05627397, 0.00841858],
       [0.00883629, 0.07880149, 0.7927754 , 0.08636829, 0.03321859]],
      dtype=float32)

In [220]:
model.evaluate(X_test, y_test_n)

32/32 [==============================] - 0s 8ms/step - loss: 0.3329 - accuracy: 0.9340


[0.3328832685947418, 0.9340000152587891]